In [1]:
import itertools
import functools

import numpy as np
from scipy.optimize import minimize
from dendropy import Tree,  DnaCharacterMatrix
from dendropy.simulate import treesim
from beaglepy import *
from beaglepy.utils import print_resource_list, print_flags


np.random.seed(0)

# eigen decomposition for the JC69 model
evec = np.array([
1.0, 2.0, 0.0, 0.5, 1.0, -2.0, 0.5, 0.0, 1.0, 2.0, 0.0, -0.5, 1.0, -2.0, -0.5, 0.0
])

ivec = np.array([
0.25, 0.25, 0.25, 0.25, 0.125, -0.125, 0.125, -0.125, 0.0, 1.0, 0.0,
-1.0, 1.0, 0.0, -1.0, 0.0
])

evalues = np.array(
[0.0, -1.3333333333333333, -1.3333333333333333, -1.3333333333333333])

freqs = np.full((4,), 0.25)

# simulate an alignment of length seq_len from a tree
def simulate(tree, seq_len, alphabet='ACTG'):
    edge_lengths = np.empty(len(tree.taxon_namespace) * 2 - 2)
    for node in tree.postorder_node_iter():
        if node != tree.seed_node:
            edge_lengths[node.index] = node.edge_length
    edge_lengths /= np.sum(edge_lengths)

    mats = JC69_p_t(np.expand_dims(edge_lengths, axis=1))
    
    alignment = [None] *(len(tree.taxon_namespace)*2-1)
    alignment[tree.seed_node.index] = np.random.choice(np.arange(4), seq_len, p=freqs)
    for node in tree.preorder_node_iter():
        if node != tree.seed_node:
            node.edge_length = edge_lengths[node.index]
            sequence = []
            for i in range(seq_len):
                probs = mats[node.index][alignment[node.parent_node.index][i],]
                sequence.append(np.random.choice(np.arange(len(alphabet)), 1, p=probs))
            alignment[node.index] = np.concatenate(sequence)
    alignment = {taxon.label: ''.join(list(map(lambda x: alphabet[x], seq))) for taxon, seq in zip(tree.taxon_namespace, alignment[:len(tree.taxon_namespace)])}
    return alignment

    
def JC69_p_t(d):
    return evec.reshape((4, 4)) @ (np.expand_dims(np.exp(evalues*d), axis=-1)*np.eye(4)) @ ivec.reshape((4, 4))

# Simulate tree and alignment

In [2]:
tree = treesim.birth_death_tree(birth_rate=1.0, death_rate=0.5, num_extant_tips=100)

s = len(tree.taxon_namespace)

postorder_indices = []
edge_lengths = np.empty(len(tree.taxon_namespace) * 2 - 2)
for node in tree.postorder_node_iter():
    if not node.is_leaf():
        node.index = s
        s += 1
        children = node.child_nodes()
        postorder_indices.append((node.index, children[0].index, children[1].index))
    else:
        node.index = tree.taxon_namespace.accession_index(node.taxon)
    node.annotations.add_bound_attribute("index")
    if node != tree.seed_node:
        edge_lengths[node.index] = node.edge_length

seq_length = 1000
sequences = simulate(tree, seq_length)
alignment = DnaCharacterMatrix.from_dict(sequences)

# Tree likelihood with numpy

In [3]:
def calculate_log_prob(partials, weights, postorder_indices, edge_lengths):
    bls = np.expand_dims(edge_lengths, axis=1)
    mats = JC69_p_t(bls)

    for node, left, right in postorder_indices:
        partials[node] = np.matmul(mats[left], partials[left]) * np.matmul(mats[right], partials[right])
    return np.sum(np.log(np.matmul(freqs, partials[postorder_indices[-1][0]])) * weights)

dna_map = {
    "A": [1.0, 0.0, 0.0, 0.0],
    "C": [0.0, 1.0, 0.0, 0.0],
    "G": [0.0, 0.0, 1.0, 0.0],
    "T": [0.0, 0.0, 0.0, 1.0],
}

weights = np.ones(seq_length)
partials = np.empty((len(alignment)*2 - 1, 4, seq_length))
partials_beagle = []

for idx, seq in enumerate(sequences.values()):
    partial = [dna_map[nuc] for nuc in seq]
    partials_beagle.append(list(itertools.chain.from_iterable(partial)))
    partials[idx,] = np.transpose(np.array(partial))

log_prob = functools.partial(calculate_log_prob, partials, weights, postorder_indices)

In [4]:
%%time
lp_numpy = log_prob(edge_lengths)

CPU times: user 4 ms, sys: 1.87 ms, total: 5.87 ms
Wall time: 4.77 ms


In [5]:
print(lp_numpy)

-83662.60715657224


In [6]:
%%time
neg_log_prob = lambda log_edge_lengths: -log_prob(np.concatenate((np.exp(log_edge_lengths)+1e-8, np.zeros(1))))
res_numpy = minimize(neg_log_prob, np.log(edge_lengths[:-1]+1e-8), method='powell',
               options={'ftol': 0.0000001, 'maxiter': 10})

CPU times: user 29.6 s, sys: 115 ms, total: 29.7 s
Wall time: 29.8 s


In [7]:
lp_ml = log_prob(np.concatenate((np.exp(res_numpy.x)+1e-8, np.zeros(1))))
print('NUMPY ML: ', lp_ml)

NUMPY ML:  -11285.771667028552


# Tree likelihood with BEAGLE

In [8]:
print_resource_list()

Available resources:
	Resource 0:
		Name : CPU
		Desc : 
		Flags: PROCESSOR_CPU PRECISION_DOUBLE PRECISION_SINGLE COMPUTATION_SYNCH EIGEN_REAL EIGEN_COMPLEX SCALING_MANUAL SCALING_AUTO SCALING_ALWAYS SCALING_DYNAMIC SCALERS_RAW SCALERS_LOG VECTOR_NONE VECTOR_SSE THREADING_NONE FRAMEWORK_CPU



In [9]:
sequence_count = len(alignment)
buffer_count = 2 * sequence_count - 1
matrix_count = 2 * sequence_count - 2
scale_count = 0

# create an instance of the BEAGLE library
instance, instance_details = create_instance(
    sequence_count,  # Number of tip data elements (input)
    buffer_count,  # Number of partials buffers to create (input)
    0,  # Number of compact state representation buffers to create (input)
    4,  # Number of states in the continuous-time Markov chain (input)
    seq_length,  # Number of site patterns to be handled by the instance (input)
    1,  # Number of rate matrix eigen-decomposition buffers to allocate (input)
    matrix_count,  # Number of rate matrix buffers (input)
    1,  # Number of rate categories (input)
    scale_count,  # Number of scaling buffers
    None,  # List of potential resource on which this instance is allowed (input, NULL implies no restriction
    0,  # Bit-flags indicating preferred implementation charactertistics, see BeagleFlags (input)
    0,  # Bit-flags indicating required implementation characteristics, see BeagleFlags (input)
)

if instance < 0:
    print('Failed to obtain BEAGLE instance')
else:
    print("Using resource {}:".format(instance_details.resource_number))
    print("\tRsrc Name : {}".format(instance_details.resource_name))
    print("\tImpl Name : {}".format(instance_details.impl_name))
    print("\tImpl Desc : {}".format(instance_details.impl_description))
    print("\tFlags:", end='')
    print_flags(instance_details.flags)

Using resource 0:
	Rsrc Name : CPU
	Impl Name : CPU-4State-SSE-Double
	Impl Desc : none
	Flags: PROCESSOR_CPU PRECISION_DOUBLE COMPUTATION_SYNCH EIGEN_REAL SCALING_MANUAL SCALERS_RAW VECTOR_SSE THREADING_NONE FRAMEWORK_CPU

In [10]:
for idx, partial in enumerate(partials_beagle):
    set_tip_partials(instance, idx, partial)

set_eigen_decomposition(instance, 0, evec, ivec, evalues)
set_state_frequencies(instance, 0, freqs)
set_category_weights(instance, 0, np.array([1.0]))
set_pattern_weights(instance, weights)
_ = set_category_rates(instance, [1.0])

In [11]:
def calculate_log_prob_beagle(instance, postorder_indices, edge_lengths):
    update_transition_matrices(
                instance,  # instance
                0,  # eigenIndex
                list(range(len(edge_lengths))),  # probabilityIndices
                None,  # firstDerivativeIndices
                None,  # secondDerivativeIndices
                edge_lengths)  # edgeLengths

    operations = []
    for node, left, right in postorder_indices:
        operations.append(BeagleOperation(node, BEAGLE_OP_NONE,
                                BEAGLE_OP_NONE, left, left, right, right))
    update_partials(
                instance,
                operations,
                BEAGLE_OP_NONE)

    lp = np.empty(1)
    returnCode = calculate_root_log_likelihoods(
            instance,  # instance
            postorder_indices[-1][:1],  # bufferIndices
            [0],  # weights
            [0],  # stateFrequencies
            [BEAGLE_OP_NONE],  # cumulative scaling index
            1,  # count
            lp)
    return lp.copy()[0]

log_prob_beagle = functools.partial(calculate_log_prob_beagle, instance, postorder_indices)

In [12]:
%%time
lp_beagle = log_prob_beagle(edge_lengths)

CPU times: user 1.43 ms, sys: 741 µs, total: 2.17 ms
Wall time: 2.18 ms


In [13]:
print('BEAGLE log prob:', lp_beagle)
print('NUMPY  log prob:', lp_numpy)

BEAGLE log prob: -83662.60715657199
NUMPY  log prob: -83662.60715657224


In [14]:
%%time
neg_log_prob = lambda log_edge_lengths: -log_prob_beagle(np.concatenate((np.exp(log_edge_lengths)+1e-8, np.zeros(1))))
res = minimize(neg_log_prob, np.log(edge_lengths[:-1]+1e-8), method='powell',
               options={'ftol': 0.0000001, 'maxiter': 10})

CPU times: user 26.5 s, sys: 287 ms, total: 26.8 s
Wall time: 26.6 s


In [15]:
lp_beagle_ml = log_prob_beagle(np.concatenate((np.exp(res.x)+1e-8, np.zeros(1))))
print('BEAGLE ML: ', lp_beagle_ml, log_prob_beagle(np.concatenate((np.exp(res_numpy.x)+1e-8, np.zeros(1)))))
print('NUMPY ML:  ', lp_ml, log_prob(np.concatenate((np.exp(res.x)+1e-8, np.zeros(1)))))

BEAGLE ML:  -10415.852968783953 -11285.771666926767
NUMPY ML:   -11285.771667028552 -10415.852968794894


In [16]:
finalize_instance(instance);